In [1]:
%matplotlib inline

from scipy.special import logsumexp
import numpy as np
import itertools
import torch

import matplotlib
import matplotlib.pyplot as plt
import seaborn

seaborn.set_context("talk")
torch.set_printoptions(precision=4)
np.set_printoptions(precision=4)

In [ ]:
class P1:
    def __init__(self, fname):
        self.x = np.load(fname)
        self.N, self.M, self.K = self.x.shape
                        
        # log potentials
        def edge_func(i, j):
            if i == j: 
                return 10
            elif abs(i - j) == 1: 
                return 2
            else: 
                return 0
        self.edge_theta = np.fromfunction(np.vectorize(edge_func), (self.K, self.K))
        self.node_theta = 10 * self.x
        
        # MF
        mu = np.zeros(self.x.shape) # mu[i, j, k] = q_ij(y_ij = k)
        
        # LBP
        msgs = None
        prev_msgs = None
        bels = None
    
    ### helper functions
    
    def softmax(self, x):
        # normalizes vector x
        e_x = np.exp(x - np.max(x))
        return e_x / e_x.sum()

    def neighbors(self, i, j):
        out = []
        for (di, dj) in [(0, -1), (-1, 0), (0, 1), (1, 0)]:
            i_, j_ = i + di, j + dj
            if 0 <= i_ < self.N and 0 <= j_ < self.M and (i_, j_) != (i, j):
                out.append((i_, j_))
        return out

    def nodes(self): 
        return itertools.product(range(self.N), range(self.M))
    
    def dir_edges(self):
        for t in self.nodes():
            for n in self.neighbors(*t):
                yield (t, n)

    def from_one_hot(self, one_hot_vec):
        return np.where(one_hot_vec)[0][0]
                
    ### inference
    
    def brute_force(self):
        def unnormed_log_joint(y):
            p = sum([self.node_theta[t].dot(y[t]) for t in self.nodes()])
            p += 0.5 * sum(( 
                self.edge_theta[self.from_one_hot(y[n]), self.from_one_hot(y[t])]
                for (n, t) in self.dir_edges() 
            ))
            return p
        
        def all_ys(key=None):
            tuple_to_mat = lambda t: np.asarray(t).reshape((self.N, self.M))
            to_one_hot = lambda a: (a[...,None] == np.arange(self.K)).astype(float)
            arrays = map(tuple_to_mat, list(itertools.product(range(self.K), repeat=self.N*self.M)))
            sorted_arrays = sorted(map(to_one_hot, arrays), key=key)
            return itertools.groupby(sorted_arrays, key=key)
        
        marginals = np.zeros(self.x.shape)
        for t in self.nodes():
            ps = []
            for k, group in all_ys( key=lambda img: self.from_one_hot(img[t]) ):
                p = logsumexp([ unnormed_log_joint(y) for y in group ])
                ps.append(p)
            marginals[t] = self.softmax(ps) # np.exp(ps - logsumexp(ps))
        return marginals
    
    def mean_field(self, epochs=30):
        history = [self.x]
        prev_mu = self.softmax(np.zeros(self.x.shape)) 
        for epoch in range(epochs):
            mu = np.zeros(self.x.shape)
            for (i, j) in self.nodes():
                ngh = sum(( prev_mu[t] for t in self.neighbors(i, j) ))
                mu[i, j] = self.softmax(self.node_theta[i, j] + self.edge_theta.dot(ngh))
            prev_mu = mu
            history.append(prev_mu)
        return history
    
    def loopy_bp(self, epochs=30):
        history = [self.x]
        prev_msgs = {edge: np.zeros(self.K) for edge in self.dir_edges()}
        bels = np.ones(self.x.shape)
        for epoch in range(epochs):
            msgs = {edge: np.zeros(self.K) for edge in self.dir_edges()}
            for ((i, j), t) in self.dir_edges():
                for k in range(self.K):
                    ngh = lambda l: [
                        prev_msgs[(u, (i, j))][l] 
                        for u in self.neighbors(i, j) if u != t 
                    ]
                    values = [
                        self.node_theta[i, j, l] + self.edge_theta[k, l] + sum(ngh(l))
                        for l in range(self.K)
                    ]
                    msgs[((i, j), t)][k] = logsumexp(values)
            for s in self.nodes():
                ngh = (msgs[(t, s)] for t in self.neighbors(*s))
                bels[s] = self.softmax(self.node_theta[s] + sum(ngh))
            prev_msgs = msgs
            history.append(bels)
        return history

print('small')
p1 = P1('small.npy')
marginals = p1.brute_force()
print(marginals)
plt.figure()
plt.imshow(marginals)

files = ('small', ) # 'flag', 'bullseye', 'spiral'
for fname in files:
    print(fname)
    p1 = P1(fname + '.npy')
    for f in (p1.mean_field, p1.loopy_bp):
        history = f()
        for img in (history[-1], ):
            print(img)
            plt.figure()
            plt.imshow(img)

In [ ]:
"""
2.
"""
import utils
import numpy as np
import math

import torch
from torch.autograd import Variable
torch.set_printoptions(precision=4)

import seaborn
import matplotlib.pyplot as plt

class LatentLinearModel(torch.nn.Module):
    def __init__(self, K, N, J):
        super(LatentLinearModel, self).__init__()
        self.K = K
        self.N = N
        self.J = J

        self.mu_U = torch.nn.Embedding(N, K)
        self.logvar_U = torch.nn.Embedding(N, K)
        self.logvar_U.weight.data.fill_(-10)

        self.mu_V = torch.nn.Embedding(J, K)
        self.logvar_V = torch.nn.Embedding(J, K)
        self.logvar_V.weight.data.fill_(-10)

    def forward(self, users, jokes):
        batch_size = users.size()[0]
        z_U = Variable(torch.from_numpy(np.random.normal(0, 1, (batch_size, self.K))).float(), requires_grad=False)
        z_V = Variable(torch.from_numpy(np.random.normal(0, 1, (batch_size, self.K))).float(), requires_grad=False)

        u = ( z_U.mul(self.logvar_U(users).exp().sqrt()).add(self.mu_U(users)) ).unsqueeze(1)
        v = ( z_V.mul(self.logvar_V(jokes).exp().sqrt()).add(self.mu_V(jokes)) ).unsqueeze(2)
        r = torch.bmm(u, v).squeeze()
        return r

def vi_loss(model, users, jokes):
    batch_size = users.size()[0]
    var_for_prior = Variable(
        torch.from_numpy(5 * np.ones((batch_size, model.K))).float(),
        requires_grad=False
    )
    loss = (0.5 * (var_for_prior.sqrt() - model.logvar_U(users))
        + (model.logvar_U(users).exp() + model.mu_U(users) ** 2) / 10).sum()
    loss += (0.5 * (var_for_prior.sqrt() - model.logvar_V(jokes))
        + (model.logvar_V(jokes).exp() + model.mu_V(jokes) ** 2) / 10).sum()

    return loss

Ks = range(10, 11)

for K in Ks:
    train_logprobs = []
    train_lower_bound = []
    test_logprobs = []
    test_loss = []

    # Load data iterators
    train_iter, val_iter, test_iter = utils.load_jester(
        batch_size=100, subsample_rate=0.1, load_text=False)

    # Construct our model by instantiating the class defined above
    N, J = 70000, 150
    model = LatentLinearModel(K, N, J)

    # Construct our loss function and an Optimizer. The call to model.parameters()
    # in the SGD constructor will contain the learnable parameters of the two
    # nn.Linear modules which are members of the model.
    criterion = torch.nn.MSELoss(size_average=False)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.04)

    # Optimize model parameters
    num_epochs = 10
    for epoch in range(num_epochs):
        lower_bound = 0
        logprob_for_epoch = 0
        n_in_epoch = 0
        U_sample = model.mu_U
        V_sample = model.mu_V

        train_iter.init_epoch()
        for batch in train_iter:
            # 0-index data
            ratings = (batch.ratings-1).float()
            users = batch.users-1
            jokes = batch.jokes-1

            n_in_epoch += ratings.size()[0]

            # Forward pass: Compute predicted y by passing x to the model
            r_pred = model(users, jokes)

            # Compute loss
            loss = 0.5 * criterion(r_pred, ratings)
            loss += vi_loss(model, users, jokes) # add entropy and prior terms
            lower_bound -= loss.data[0]

            # Update log-likelihood for epoch
            _r_pred = torch.bmm((U_sample(users)).unsqueeze(1), (V_sample(jokes)).unsqueeze(2)).squeeze()
            _logprob = - 0.5 * criterion(_r_pred, ratings) - ratings.size()[0] * np.log(1/(2 * math.pi))
            logprob_for_epoch += _logprob.data[0]

            # Zero gradients, perform a backward pass, and update the weights
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        lower_bound /= n_in_epoch
        logprob_for_epoch /= n_in_epoch
        train_logprobs.append(logprob_for_epoch)
        train_lower_bound.append(lower_bound)

        # Predict on test set
        test_logprob_for_epoch = 0
        n = 0
        for batch in test_iter:
            # 0-index data
            ratings = (batch.ratings-1).float()
            users = batch.users-1
            jokes = batch.jokes-1

            n += ratings.size()[0]

            # Forward pass: Compute predicted y by passing x to the model
            r_pred = model(users, jokes)

            # Update log-likelihood for epoch
            _r_pred = torch.bmm((U_sample(users)).unsqueeze(1), (V_sample(jokes)).unsqueeze(2)).squeeze()
            _logprob = - 0.5 * criterion(_r_pred, ratings) - ratings.size()[0] * np.log(1/(2 * math.pi))
            test_logprob_for_epoch += _logprob.data[0]

        test_logprob_for_epoch /= n
        test_logprobs.append(test_logprob_for_epoch)

        # Print summary for epoch
        print(epoch, logprob_for_epoch, lower_bound)

    plt.figure()
    plt.plot(range(num_epochs), train_logprobs, c='r')
    plt.xlabel("Epoch")
    plt.ylabel('Training Log-likelihood')

    plt.figure()
    plt.plot(range(num_epochs), test_logprobs, c='b')
    plt.xlabel("Epoch")
    plt.ylabel('Testing Log-likelihood')

    plt.figure()
    plt.plot(range(num_epochs), train_lower_bound, c='g')
    plt.xlabel("Epoch")
    plt.ylabel('Training Lower Bound')

    plt.show()